In [4]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import kagglehub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout


dataset_path = kagglehub.dataset_download("ismailnasri20/driver-drowsiness-dataset-ddd")


data_path = os.path.join(dataset_path, "Driver Drowsiness Dataset (DDD)")


if not os.path.exists(data_path):
    raise FileNotFoundError(f"Path {data_path} not found!")

print("Files and folders in data_path:", os.listdir(data_path))


classes = ["Drowsy", "Non Drowsy"]
class_mapping = {"Drowsy": 1, "Non Drowsy": 0}  # 1 = Drowsy, 0 = Awake

frame_skip = 10


img_size = (32, 32)

def extract_frames(video_path, frame_skip=frame_skip):
    frames = []
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Unable to open video {video_path}")
        return frames

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            frame = cv2.resize(frame, img_size) / 255.0  # تصغير وتطبيع الصورة
            frames.append(frame)

        frame_count += 1

    cap.release()
    return frames


def load_video_data(data_path):
    X, y = [], []

    for class_name in classes:
        class_path = os.path.join(data_path, class_name)
        if not os.path.exists(class_path):
            print(f"Warning: Folder {class_name} not found, skipping...")
            continue

        for video_name in os.listdir(class_path):
            video_path = os.path.join(class_path, video_name)
            frames = extract_frames(video_path, frame_skip)

            if not frames:
                print(f"Warning: No frames extracted from {video_path}")
                continue

            X.append(frames)
            y.append(class_mapping[class_name])

    return np.array(X), np.array(y)


X, y = load_video_data(data_path)

if X.shape[0] == 0:
    raise ValueError("No data found! Check the dataset path and files.")

print("Data shape after loading:", X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data successfully split!")
print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

if len(X_train) == 0 or len(y_train) == 0:
    raise ValueError("Insufficient training data!")


model = Sequential([
    TimeDistributed(Conv2D(16, (3,3), activation='relu', padding='same'), input_shape=(10, 32, 32, 3)),
    TimeDistributed(MaxPooling2D((2,2))),
    TimeDistributed(Conv2D(32, (3,3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2,2))),
    TimeDistributed(Flatten()),

    LSTM(32, return_sequences=True),
    LSTM(16),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3
batch_size = 8


history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))


loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.2f}")

Files and folders in data_path: ['Drowsy', 'Non Drowsy']
Data shape after loading: (41793, 1, 32, 32, 3) (41793,)
Data successfully split!
Training samples: 33434
Testing samples: 8359
Epoch 1/3
4180/4180 ━━━━━━━━━━━━━━━━━━━━ 91s 20ms/step - accuracy: 0.9019 - loss: 0.2020 - val_accuracy: 0.9987 - val_loss: 0.0048
Epoch 2/3
4180/4180 ━━━━━━━━━━━━━━━━━━━━ 140s 20ms/step - accuracy: 0.9975 - loss: 0.0093 - val_accuracy: 0.9981 - val_loss: 0.0056
Epoch 3/3
4180/4180 ━━━━━━━━━━━━━━━━━━━━ 136s 18ms/step - accuracy: 0.9985 - loss: 0.0059 - val_accuracy: 0.9951 - val_loss: 0.0139
262/262 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9951 - loss: 0.0148
Test accuracy: 1.00
